Het begin van wat ik gevolgd heb van https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24 is nog wel solid. Later gaat er iets niet goed, dus dat moet nog even gefixt worden, maar volgens mij is dit al semi de basis van onze recommender. 

In [2]:
import json
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np

import re, string, unicodedata
import nltk
#import contractions
import inflect
import sklearn
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from gensim import corpora, models
from pprint import pprint

In [3]:
comment_dict = {}

# filtering data
for line in open('RC_2015-01/test_data.txt'):
    line_short = line[:-2]
    json_file = json.loads(line_short)
    comment_id =  json_file['id']
    comment_body = json_file['body']
    comment_sub = json_file['subreddit']
    comment = {"body": comment_body, "subreddit": comment_sub}
    comment_dict[comment_id] = comment


#writing filtered data to file
with open('result.json', 'w') as fp:
    json.dump(comment_dict, fp)
    
#sanity check
with open('result.json', 'r') as rf:
    data = json.load(rf)
    for entry in data:
        print(data[entry])
    

JSONDecodeError: Expecting ',' delimiter: line 1 column 490 (char 489)

In [3]:
#Copied from Manon's Text Mining project

def remove_non_ascii(words):
    #"""Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    #"""Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    #"""Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    #"""Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    #"""Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    #"""Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    #"""Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

def preprocess(words):
    normalize(words)
    stem_words(words)
    lemmatize_verbs(words)
    return words


In [1]:
all_words = []

# Pre-processing sanity check (words are weird)
with open('result.json', 'r') as rf:
    data = json.load(rf)
    for entry in data:
        doc_sample = data[entry]['body']
       # print('original document: ')
        words = []
        for word in doc_sample.split(' '):
            words.append(word)
      #  print(words)
        
        words = normalize(words)
        words = stem_words(words)
        words = lemmatize_verbs(words)
      #  print('\n\n tokenized and lemmatized document: ')
      #  print(words)
        
        all_words.append(words)
print(all_words[:10])


FileNotFoundError: [Errno 2] No such file or directory: 'result.json'

In [5]:
        
dictionary = gensim.corpora.Dictionary(all_words)
        
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
                
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in all_words]
bow_corpus[28]



0 quarry
1 

2 im
3 saidhttpimgurcom9ttainh
4 salut
5 sur
6 basebal
7 bite
8 cain
9 crazy
10 get


[(4, 1), (12, 1), (85, 2), (152, 1), (162, 1), (163, 1), (164, 1), (165, 1)]

In [6]:
bow_doc_28 = bow_corpus[28]
for i in range(len(bow_doc_28)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_28[i][0], 
                                               dictionary[bow_doc_28[i][0]], 
bow_doc_28[i][1]))

Word 4 ("get") appears 1 time.
Word 12 ("fuck") appears 1 time.
Word 85 ("good") appears 2 time.
Word 152 ("pretty") appears 1 time.
Word 162 ("job") appears 1 time.
Word 163 ("laugh") appears 1 time.
Word 164 ("lol") appears 1 time.
Word 165 ("wel") appears 1 time.


In [7]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf:
    pprint(doc)
    break
    
# Dit doet dus nie, weet niet waarom

[]


In [8]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"thread" + 0.019*"ev" + 0.018*"bot" + 0.018*"act" + 0.016*"post" + 0.015*"yo" + 0.015*"lik" + 0.015*"pleas" + 0.015*"fuck" + 0.013*"bite"
Topic: 1 
Words: 0.032*"good" + 0.030*"say" + 0.023*"peopl" + 0.022*"us" + 0.019*"that" + 0.018*"lov" + 0.018*"man" + 0.014*"iv" + 0.014*"get" + 0.013*"form"
Topic: 2 
Words: 0.060*"see" + 0.038*"let" + 0.036*"goe" + 0.035*"deep" + 0.035*"hol" + 0.035*"rabbit" + 0.028*"bal" + 0.023*"day" + 0.023*"peopl" + 0.018*"wel"
Topic: 3 
Words: 0.029*"lik" + 0.023*"didnt" + 0.021*"get" + 0.019*"thing" + 0.016*"

i" + 0.015*"think" + 0.015*"mak" + 0.014*"would" + 0.013*"want" + 0.012*"tim"
Topic: 4 
Words: 0.054*"delet" + 0.030*"know" + 0.028*"dont" + 0.023*"kid" + 0.021*"team" + 0.019*"on" + 0.019*"want" + 0.015*"play" + 0.015*"understand" + 0.014*"im"
Topic: 5 
Words: 0.035*"get" + 0.026*"polit" + 0.019*"lov" + 0.016*"gam" + 0.015*"us" + 0.015*"peopl" + 0.014*"part" + 0.014*"don" + 0.013*"due" + 0.012*"on"
Topic: 6 
Words: 0.046*"lik" + 

In [9]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.019*"lik" + 0.015*"say" + 0.015*"lol" + 0.014*"lov" + 0.011*"yo" + 0.011*"see" + 0.010*"fuck" + 0.009*"what" + 0.009*"get" + 0.009*"feel"
Topic: 1 Word: 0.014*"think" + 0.012*"that" + 0.011*"ev" + 0.008*"stat" + 0.008*"would" + 0.008*"tru" + 0.007*"unit" + 0.007*"bas" + 0.007*"dont" + 0.007*"id"
Topic: 2 Word: 0.151*"delet" + 0.087*"remov" + 0.015*"ye" + 0.009*"lik" + 0.008*"want" + 0.007*"on" + 0.006*"good" + 0.006*"suck" + 0.006*"put" + 0.005*"peopl"
Topic: 3 Word: 0.022*"delet" + 0.016*"go" + 0.012*"know" + 0.012*"look" + 0.011*"good" + 0.010*"post" + 0.010*"oh" + 0.010*"dont" + 0.009*"get" + 0.008*"thing"
Topic: 4 Word: 0.012*"lik" + 0.011*"get" + 0.011*"two" + 0.011*"im" + 0.010*"on" + 0.009*"spot" + 0.008*"year" + 0.008*"gam" + 0.008*"us" + 0.008*"tim"
Topic: 5 Word: 0.015*"much" + 0.015*"play" + 0.012*"us" + 0.010*"dont" + 0.010*"sorry" + 0.009*"ok" + 0.009*"im" + 0.009*"day" + 0.009*"also" + 0.009*"try"
Topic: 6 Word: 0.037*"rabbit" + 0.037*"let" + 0.037*"deep"

In [10]:
for index, score in sorted(lda_model[bow_corpus[28]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.589565634727478	 
Topic: 0.032*"good" + 0.030*"say" + 0.023*"peopl" + 0.022*"us" + 0.019*"that" + 0.018*"lov" + 0.018*"man" + 0.014*"iv" + 0.014*"get" + 0.013*"form"

Score: 0.19771523773670197	 
Topic: 0.035*"get" + 0.026*"polit" + 0.019*"lov" + 0.016*"gam" + 0.015*"us" + 0.015*"peopl" + 0.014*"part" + 0.014*"don" + 0.013*"due" + 0.012*"on"

Score: 0.14270007610321045	 
Topic: 0.060*"see" + 0.038*"let" + 0.036*"goe" + 0.035*"deep" + 0.035*"hol" + 0.035*"rabbit" + 0.028*"bal" + 0.023*"day" + 0.023*"peopl" + 0.018*"wel"

Score: 0.010003894567489624	 
Topic: 0.029*"lik" + 0.023*"didnt" + 0.021*"get" + 0.019*"thing" + 0.016*"

i" + 0.015*"think" + 0.015*"mak" + 0.014*"would" + 0.013*"want" + 0.012*"tim"

Score: 0.010003355331718922	 
Topic: 0.021*"sav" + 0.020*"would" + 0.018*"id" + 0.017*"ev" + 0.015*"im" + 0.014*"stat" + 0.014*"gt" + 0.014*"gam" + 0.013*"on" + 0.012*"unit"

Score: 0.010003204457461834	 
Topic: 0.021*"thread" + 0.019*"ev" + 0.018*"bot" + 0.018*"act" + 0.016*"po

In [11]:
unseen_document = ['How', 'a', 'Pentagon', 'deal', 'became','an', 'identity', 'crisis', 'for', 'Google'] 
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5499863028526306	 Topic: 0.032*"good" + 0.030*"say" + 0.023*"peopl" + 0.022*"us" + 0.019*"that"
Score: 0.05000265687704086	 Topic: 0.029*"lik" + 0.023*"didnt" + 0.021*"get" + 0.019*"thing" + 0.016*"

i"
Score: 0.05000166967511177	 Topic: 0.021*"thread" + 0.019*"ev" + 0.018*"bot" + 0.018*"act" + 0.016*"post"
Score: 0.05000150203704834	 Topic: 0.046*"lik" + 0.031*"us" + 0.029*"remov" + 0.024*"go" + 0.020*"see"
Score: 0.05000144615769386	 Topic: 0.035*"get" + 0.026*"polit" + 0.019*"lov" + 0.016*"gam" + 0.015*"us"
Score: 0.05000137537717819	 Topic: 0.023*"point" + 0.020*"lik" + 0.020*"play" + 0.018*"real" + 0.017*"get"
Score: 0.050001367926597595	 Topic: 0.054*"delet" + 0.030*"know" + 0.028*"dont" + 0.023*"kid" + 0.021*"team"
Score: 0.05000133439898491	 Topic: 0.060*"see" + 0.038*"let" + 0.036*"goe" + 0.035*"deep" + 0.035*"hol"
Score: 0.050001226365566254	 Topic: 0.032*"gam" + 0.027*"win" + 0.024*"pick" + 0.022*"get" + 0.022*"on"
Score: 0.05000109225511551	 Topic: 0.021*"sav" + 0.